<a href="https://colab.research.google.com/github/prabhjotschugh/Fish-Detection-YOLOv8/blob/main/Fine_Tuning_YOLOv8_(custom_data).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'fishcustomdataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5514463%2F9132940%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240809%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240809T051238Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D19cf2332e9106bc1f6605fb2b6358152504ce7ede1cf9085321d09037e20502c491ad72ec7d5304c908e4750585b5db03719d9a107d1483e1553d0a30471106a32ab0066d8d446af84e6ad7fe5d24c4f0192c490db82763294f36b751c5f24c5d8658ef2d724ccea17ceb7a436153f98d90da4fc7fc8bff7a3f15b756cbcc6f2e8088066ff883140105ee0fe53d5a1980371d0145f52bc47f14153e6d5d3d7c8b94813eff6f895428d6854c87b7fcd1e8f9e3c91afb71975ae170be2e57d761e23d923513f33fefb6dc0b15330af84625425167f698d6a1d5e1e6ace081e04de7b867f646022325cac6cb86a255b9f0f3a6c87ad3e7ad49c8bd6cad7306601e1'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 617.1 kB/s eta 0:00:000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.6/865.6 kB 6.5 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from ultralytics import YOLO

# Load the model.
model = YOLO('yolov8l.pt')

# Training.
results = model.train(
   data='/kaggle/input/fishcustomdataset/Dataset/data_custom.yaml',
   imgsz=640,
   epochs=300,
   batch=8,
   name='yolov8l_v8_300e'
)

100%|██████████| 83.7M/83.7M [00:00<00:00, 275MB/s]


Ultralytics YOLOv8.2.75 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=/kaggle/input/fishcustomdataset/Dataset/data_custom.yaml, epochs=300, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8l_v8_300e, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_

100%|██████████| 755k/755k [00:00<00:00, 13.7MB/s]
2024-08-09 04:46:47,385	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-08-09 04:46:48,876	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  3    279808  ultralytics.nn.modules.block.C2f             [128, 128, 3, True]           
  3                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  4                  -1  6   2101248  ultralytics.nn.modules.block.C2f             [256, 256, 6, True]           
  5                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  6                  -1  6   8396800  ultralytics.nn.modules.block.C2f             [512, 512, 6, True]           
  7                  -1  1   2360320  ultralytics

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

...

                   all         10         12      0.893      0.706      0.874      0.766
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 147, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



247 epochs completed in 0.339 hours.
Optimizer stripped from runs/detect/yolov8l_v8_300e/weights/last.pt, 87.7MB
Optimizer stripped from runs/detect/yolov8l_v8_300e/weights/best.pt, 87.7MB

Validating runs/detect/yolov8l_v8_300e/weights/best.pt...
Ultralytics YOLOv8.2.75 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 268 layers, 43,609,692 parameters, 0 gradients, 164.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  4.21it/s]


                   all         10         12      0.847      0.905      0.981      0.884
        Chinook salmon          3          3      0.811          1      0.995       0.88
        Humpback whale          2          2          1      0.889      0.995      0.895
            Orca whale          3          5          1       0.73      0.938      0.816
       Pecific herring          2          2      0.578          1      0.995      0.946
Speed: 0.2ms preprocess, 19.6ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/yolov8l_v8_300e


lr/pg0,▁▅████▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂
lr/pg1,▁▅████▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂
lr/pg2,▁▅████▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂
metrics/mAP50(B),▃▁▁▁▁▁▃▄▅▅▅▆▃▆▇▆▅█▇█▇███████▇█▇█████▇█▇▇
metrics/mAP50-95(B),▂▁▁▁▁▁▂▃▄▅▄▅▂▅▆▅▅▇▇▇▇▇▇█▇█▇█▇██▇▇███▇█▇▇
metrics/precision(B),▅▁▁▁▁▁▅▃▅▄▆▄▃▄▆▆▅▇▇▇▇▆▇██▇██▇███████▇▇▇▆
metrics/recall(B),▄▄▁▁▃▅▂▆▅▆▃▆▄▆▆▇▅▇▆█▇██▇▇▇█▇▇▇████▇█▇███
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,▅▇██▇▇▇▆▆▆▆▅▄▅▄▄▄▄▄▄▄▃▄▂▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁


In [ ]:
# Save the trained model
model.save('/kaggle/working/YOLOv8l_custom_trained_fine_tuned_model.pt')